In [19]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import pandas as pd
from torchvision.io import read_image
from torchvision import transforms
from torchvision import models

In [20]:
lr = 1e-3
batch_size = 64

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

trans = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.243, 0.262))
])

root = '' ## 이미지 경로

original_trainset = datasets.ImageFolder(root=root+"/origin-cifar-10/train", transform=trans)
original_testset = datasets.ImageFolder(root=root+"origin-cifar-10/test", transform=trans)
svd_trainset = datasets.ImageFolder(root=root+"/svd-cifar-10/train", transform=trans)
svd_testset = datasets.ImageFolder(root=root+"/svd-cifar-10/test", transform=trans)
svd4_trainset = datasets.ImageFolder(root=root+"/svd4-cifar-10/train", transform=trans)
svd4_testset = datasets.ImageFolder(root=root+"/svd4-cifar-10/test", transform=trans)

o_trainloader = DataLoader(original_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
o_testloader = DataLoader(original_testset, batch_size=batch_size, shuffle=False, drop_last=False)
s_trainloader = DataLoader(svd_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
s_testloader = DataLoader(svd_testset, batch_size=batch_size, shuffle=False, drop_last=False)
s4_trainloader = DataLoader(svd4_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
s4_testloader = DataLoader(svd4_testset, batch_size=batch_size, shuffle=False, drop_last=False)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model1 = models.resnet50(weights=models.resnet.ResNet50_Weights.DEFAULT)
model2 = models.resnet50(weights=models.resnet.ResNet50_Weights.DEFAULT)
model3 = models.resnet50(weights=models.resnet.ResNet50_Weights.DEFAULT)

model1.fc = nn.Linear(in_features=model1.fc.in_features, out_features=10, bias=True)
model2.fc = nn.Linear(in_features=model2.fc.in_features, out_features=10, bias=True)
model3.fc = nn.Linear(in_features=model3.fc.in_features, out_features=10, bias=True)

nn.init.xavier_normal_(model1.fc.weight)
nn.init.xavier_normal_(model2.fc.weight)
nn.init.xavier_normal_(model3.fc.weight)

model1.to(device)
model2.to(device)
model3.to(device)

Using cuda device


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

In [22]:
def train_loop(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset)
    loss_sum = 0
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X.to(device))
        loss = criterion(pred, y.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            loss_sum += loss
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

    return loss_sum / size

def test_loop(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    hit, loss = 0, 0
    conf_matrix = []
    for i in range(10):
        conf_matrix.append([0, 0, 0, 0])

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            loss += criterion(pred, y.to(device)).item()
            hit += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()
            temp = pred.argmax(1)
            for k in range(len(temp)):
                for i in range(10):
                    if i == temp[k]:
                        if i == y[k]:
                            conf_matrix[i][0] += 1.0
                        else:
                            conf_matrix[i][1] += 1.0
                    else:
                        if i == y[k]:
                            conf_matrix[i][2] += 1.0
                        else:
                            conf_matrix[i][3] += 1.0
    
    loss /= (size/batch_size)
    hit /= size
    print(f"Test Error: \n Accuracy: {(100*hit):>0.1f}%, Avg loss: {loss:>8f}\n")

    return conf_matrix

In [23]:
epochs = 20
train_loss1, test_res1, test_res2, test_res3 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model1.train()
    train_loss1.append(train_loop(o_trainloader, model1, criterion, optimizer1))
    model1.eval()
    test_res1.append(test_loop(o_testloader, model1, criterion))
    test_res2.append(test_loop(s_testloader, model1, criterion))
    test_res3.append(test_loop(s4_testloader, model1, criterion))

Epoch 1-------------------------------
loss: 2.418232 [    0/50000]
loss: 1.355377 [ 6400/50000]
loss: 1.277040 [12800/50000]
loss: 0.861599 [19200/50000]
loss: 0.791579 [25600/50000]
loss: 1.100408 [32000/50000]
loss: 0.749126 [38400/50000]
loss: 0.775429 [44800/50000]
Test Error: 
 Accuracy: 70.7%, Avg loss: 0.891803

Test Error: 
 Accuracy: 69.2%, Avg loss: 0.905745

Test Error: 
 Accuracy: 55.7%, Avg loss: 1.335825

Epoch 2-------------------------------
loss: 1.003006 [    0/50000]
loss: 0.615579 [ 6400/50000]
loss: 0.607499 [12800/50000]
loss: 0.690856 [19200/50000]
loss: 0.618476 [25600/50000]
loss: 0.739790 [32000/50000]
loss: 0.489801 [38400/50000]
loss: 0.842935 [44800/50000]
Test Error: 
 Accuracy: 80.1%, Avg loss: 0.598736

Test Error: 
 Accuracy: 77.0%, Avg loss: 0.681695

Test Error: 
 Accuracy: 61.3%, Avg loss: 1.169243

Epoch 3-------------------------------
loss: 0.303176 [    0/50000]
loss: 0.403631 [ 6400/50000]
loss: 0.375007 [12800/50000]
loss: 0.412683 [19200/5000

In [24]:
epochs = 20
train_loss2, test_res4, test_res5, test_res6 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model2.train()
    train_loss2.append(train_loop(s_trainloader, model2, criterion, optimizer2))
    model2.eval()
    test_res4.append(test_loop(o_testloader, model2, criterion))
    test_res5.append(test_loop(s_testloader, model2, criterion))
    test_res6.append(test_loop(s4_testloader, model2, criterion))

Epoch 1-------------------------------
loss: 2.662874 [    0/50000]
loss: 1.308737 [ 6400/50000]
loss: 0.988552 [12800/50000]
loss: 0.864831 [19200/50000]
loss: 1.072334 [25600/50000]
loss: 1.101029 [32000/50000]
loss: 0.801965 [38400/50000]
loss: 1.007986 [44800/50000]
Test Error: 
 Accuracy: 74.9%, Avg loss: 0.744969

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.730300

Test Error: 
 Accuracy: 65.1%, Avg loss: 1.010105

Epoch 2-------------------------------
loss: 0.677301 [    0/50000]
loss: 0.677508 [ 6400/50000]
loss: 0.680570 [12800/50000]
loss: 0.533542 [19200/50000]
loss: 0.681395 [25600/50000]
loss: 0.453342 [32000/50000]
loss: 0.494698 [38400/50000]
loss: 0.584859 [44800/50000]
Test Error: 
 Accuracy: 77.3%, Avg loss: 0.679132

Test Error: 
 Accuracy: 77.0%, Avg loss: 0.681713

Test Error: 
 Accuracy: 65.9%, Avg loss: 1.010048

Epoch 3-------------------------------
loss: 0.618595 [    0/50000]
loss: 0.526997 [ 6400/50000]
loss: 0.635499 [12800/50000]
loss: 0.614956 [19200/5000

In [25]:
epochs = 20
train_loss3, test_res7, test_res8, test_res9 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model3.train()
    train_loss3.append(train_loop(s4_trainloader, model3, criterion, optimizer3))
    model3.eval()
    test_res7.append(test_loop(o_testloader, model3, criterion))
    test_res8.append(test_loop(s_testloader, model3, criterion))
    test_res9.append(test_loop(s4_testloader, model3, criterion))

Epoch 1-------------------------------
loss: 2.570425 [    0/50000]
loss: 1.445567 [ 6400/50000]
loss: 0.955485 [12800/50000]
loss: 1.035960 [19200/50000]
loss: 0.945023 [25600/50000]
loss: 1.057030 [32000/50000]
loss: 0.929975 [38400/50000]
loss: 1.059534 [44800/50000]
Test Error: 
 Accuracy: 64.6%, Avg loss: 1.022390

Test Error: 
 Accuracy: 67.3%, Avg loss: 0.960465

Test Error: 
 Accuracy: 66.7%, Avg loss: 0.987490

Epoch 2-------------------------------
loss: 0.940887 [    0/50000]
loss: 0.740345 [ 6400/50000]
loss: 0.700467 [12800/50000]
loss: 0.829116 [19200/50000]
loss: 0.743836 [25600/50000]
loss: 0.966560 [32000/50000]
loss: 0.641405 [38400/50000]
loss: 0.899982 [44800/50000]
Test Error: 
 Accuracy: 71.0%, Avg loss: 0.928256

Test Error: 
 Accuracy: 73.0%, Avg loss: 0.833462

Test Error: 
 Accuracy: 71.2%, Avg loss: 0.846687

Epoch 3-------------------------------
loss: 0.526920 [    0/50000]
loss: 0.612579 [ 6400/50000]
loss: 0.820902 [12800/50000]
loss: 0.948526 [19200/5000

In [26]:
print("Result for Testing with original datasets")
print(max(test_res1), max(test_res2), max(test_res3))

print("Result for Testing with original datasets")
print(max(test_res4), max(test_res5), max(test_res6))

print("Result for Testing with original datasets")
print(max(test_res7), max(test_res8), max(test_res9))

Result for Testing with original datasets
[[891.0, 258.0, 109.0, 8742.0], [876.0, 80.0, 124.0, 8920.0], [810.0, 269.0, 190.0, 8731.0], [721.0, 439.0, 279.0, 8561.0], [760.0, 157.0, 240.0, 8843.0], [622.0, 130.0, 378.0, 8870.0], [791.0, 78.0, 209.0, 8922.0], [889.0, 206.0, 111.0, 8794.0], [880.0, 80.0, 120.0, 8920.0], [913.0, 150.0, 87.0, 8850.0]] [[861.0, 248.0, 139.0, 8752.0], [772.0, 81.0, 228.0, 8919.0], [763.0, 348.0, 237.0, 8652.0], [664.0, 518.0, 336.0, 8482.0], [721.0, 291.0, 279.0, 8709.0], [610.0, 206.0, 390.0, 8794.0], [575.0, 33.0, 425.0, 8967.0], [873.0, 295.0, 127.0, 8705.0], [871.0, 114.0, 129.0, 8886.0], [896.0, 260.0, 104.0, 8740.0]] [[712.0, 321.0, 288.0, 8679.0], [503.0, 78.0, 497.0, 8922.0], [564.0, 398.0, 436.0, 8602.0], [564.0, 815.0, 436.0, 8185.0], [713.0, 690.0, 287.0, 8310.0], [509.0, 371.0, 491.0, 8629.0], [252.0, 29.0, 748.0, 8971.0], [642.0, 128.0, 358.0, 8872.0], [856.0, 449.0, 144.0, 8551.0], [812.0, 594.0, 188.0, 8406.0]]
Result for Testing with original 

In [30]:
def best_acc_finder(test_res):
    best, max = 0, 0
    for i in range(len(test_res)):
        correct = 0.0
        total = 0.0
        for j in range(len(test_res[i])):
            correct = test_res[i][j][0] +  test_res[i][j][3]
            total = correct + test_res[i][j][2] + test_res[i][j][1]
            if max < (correct / total):
                max = correct / total
                best = i

    print(f"best accuary: {max:>0.2f} at {best}")
    return best

In [31]:
best_acc_finder(test_res1)
best_acc_finder(test_res2)
best_acc_finder(test_res3)
best_acc_finder(test_res4)
best_acc_finder(test_res5)
best_acc_finder(test_res6)
best_acc_finder(test_res7)
best_acc_finder(test_res8)
best_acc_finder(test_res9)

best accuary: 0.98 at 14
best accuary: 0.98 at 16
best accuary: 0.95 at 16
best accuary: 0.98 at 19
best accuary: 0.98 at 8
best accuary: 0.96 at 15
best accuary: 0.98 at 8
best accuary: 0.98 at 8
best accuary: 0.97 at 17


17

In [32]:
def metrics_single_class(conf_mat):
    accuracy = (conf_mat[0] + conf_mat[3]) / (conf_mat[0] + conf_mat[1] + conf_mat[2] + conf_mat[3])
    precision = conf_mat[0] / (conf_mat[0] + conf_mat[1])
    recall = conf_mat[0] / (conf_mat[0] + conf_mat[2])
    f1_score = 2 * precision * recall / (precision + recall)
    
    return accuracy, precision, recall, f1_score

def metrics(conf_mats):
    sum_acc, sum_pre, sum_recall, sum_f1 = 0.0, 0.0, 0.0, 0.0
    for i in range(len(conf_mats)):
        temp_acc, temp_pre, temp_recall, temp_f1 = metrics_single_class(conf_mats[i])
        #print(f"For class-{i}:\n Accuracy: {temp_acc:>0.4f}, Precision: {temp_pre:>0.4f}, Recall: {temp_recall:>0.4f}, F1-score: {temp_f1:>0.4f}")
        sum_acc += temp_acc
        sum_pre += temp_pre
        sum_recall += temp_recall
        sum_f1 += temp_f1

    sum_acc /= 10
    sum_pre /= 10
    sum_recall /= 10
    sum_f1 /= 10

    return sum_acc, sum_pre, sum_recall, sum_f1


print(metrics(test_res1[best_acc_finder(test_res1)]))
print(metrics(test_res2[best_acc_finder(test_res2)]))
print(metrics(test_res3[best_acc_finder(test_res3)]))
print(metrics(test_res4[best_acc_finder(test_res4)]))
print(metrics(test_res5[best_acc_finder(test_res5)]))
print(metrics(test_res6[best_acc_finder(test_res6)]))
print(metrics(test_res7[best_acc_finder(test_res7)]))
print(metrics(test_res8[best_acc_finder(test_res8)]))
print(metrics(test_res9[best_acc_finder(test_res9)]))

best accuary: 0.98 at 14
(0.9653, 0.8296794298094333, 0.8265, 0.8269203951828071)
best accuary: 0.98 at 16
(0.9545199999999999, 0.7826007477946206, 0.7726, 0.7746791582146295)
best accuary: 0.95 at 16
(0.9229200000000001, 0.6712399467604688, 0.6145999999999999, 0.6133574540872406)
best accuary: 0.98 at 19
(0.96472, 0.8258310831679866, 0.8235999999999999, 0.8237786149548822)
best accuary: 0.98 at 8
(0.95734, 0.7864677381535649, 0.7867000000000001, 0.7807146671894369)
best accuary: 0.96 at 15
(0.9390000000000001, 0.7268082722512336, 0.6950000000000001, 0.6981558167402382)
best accuary: 0.98 at 8
(0.9507999999999999, 0.7594597036368742, 0.7540000000000001, 0.7483880924195984)
best accuary: 0.98 at 8
(0.9539200000000001, 0.7703511451202903, 0.7696, 0.7657774720834494)
best accuary: 0.97 at 17
(0.9494999999999999, 0.7513386829324096, 0.7474999999999999, 0.7488944905407856)
